Fossil Fuel Power Plants vs Renewable Power Plants

Comparing environmental(AQI), health (Chronic Resp) and economic data (CAINC1) by county.



- Compare Towns with Nuclear vs Towns without Nuclear
- Compare Towns with renewables within themselves
- Compare Towns with a power plant vs the rest of the state

Compare years 2000 and 2010


## Documentation

Power Plant Data: https://www.kaggle.com/datasets/behroozsohrabi/us-electric-power-plants/data

Resp Data: https://ghdx.healthdata.org/sites/default/files/record-attached-files/IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014_NATIONAL_XLSX.zip

AQI Data: https://aqs.epa.gov/aqsweb/airdata/annual_aqi_by_county_2010.zip, https://aqs.epa.gov/aqsweb/airdata/annual_aqi_by_county_2000.zip

Cainc1 Data: https://apps.bea.gov/regional/zip/CAINC1.zip

## Plan of Attack:
- Bring in all relevant data
- Filter for study population
    - Counties with Power Plants
- Join/Merge the different datasets into one master dataset for analysis
- Compare towns with power plants to towns without power plants
- Compare towns with different types of power plants
- Compare different types of renewables
- Discuss any confounding variables
- 

In [92]:

## Imports and constants
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

us_state_to_abbrev = {
    "ALABAMA": "AL",
    "ALASKA": "AK",
    "ARIZONA": "AZ",
    "ARKANSAS": "AR",
    "CALIFORNIA": "CA",
    "COLORADO": "CO",
    "CONNECTICUT": "CT",
    "DELAWARE": "DE",
    "FLORIDA": "FL",
    "GEORGIA": "GA",
    "HAWAII": "HI",
    "IDAHO": "ID",
    "ILLINOIS": "IL",
    "INDIANA": "IN",
    "IOWA": "IA",
    "KANSAS": "KS",
    "KENTUCKY": "KY",
    "LOUISANA": "LA",
    "MAINE": "ME",
    "MARYLAND": "MD",
    "MASSACHUSETTS": "MA",
    "MICHIGAN": "MI",
    "MINNESOTA": "MN",
    "MISSISSIPPI": "MS",
    "MISSOURI": "MO",
    "MONTANA": "MT",
    "NEBRASKA": "NE",
    "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM",
    "NEW YORK": "NY",
    "NORTH CAROLINA": "NC",
    "NORTH DAKOTA": "ND",
    "OHIO": "OH",
    "OKLAHOMA": "OK",
    "OREGON": "OR",
    "PENNSYLVANIA": "PA",
    "RHODE ISLAND": "RI",
    "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN",
    "TEXAS": "TX",
    "UTAH": "UT",
    "VERMONT": "VT",
    "VIRGINIA": "VA",
    "WASHINGTON": "WA",
    "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI",
    "WYOMING": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [78]:
us_state_to_abbrev.keys()

dict_keys(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'District of Columbia', 'American Samoa', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'United States Minor Outlying Islands', 'U.S. Virgin Islands'])

In [43]:
## Init Power plant df (power_df)
power_df = pd.read_csv("data/Power_Plants.csv")
power_df = power_df[['CITY', 'STATE', 'COUNTY', 'NAICS_DESC', 'OPER_CAP', 'SUMMER_CAP', 'WINTER_CAP', 'PLAN_CAP',
       'RETIRE_CAP', 'GEN_UNITS', 'PLAN_UNIT', 'RETIR_UNIT', 'PRIM_FUEL',
       'SEC_FUEL', 'COAL_USED', 'NGAS_USED', 'OIL_USED', 'NET_GEN']] 
power_df.head()

,CITY,STATE,COUNTY,NAICS_DESC,OPER_CAP,SUMMER_CAP,WINTER_CAP,PLAN_CAP,RETIRE_CAP,GEN_UNITS,PLAN_UNIT,RETIR_UNIT,PRIM_FUEL,SEC_FUEL,COAL_USED,NGAS_USED,OIL_USED,NET_GEN
0,SAND POINT,AK,ALEUTIANS EAST,FOSSIL FUEL ELECTRIC POWER GENERATION,4.0,1.8,1.7,0.0,0.0,6,0,0,DFO,WND,0,0,0,347.00
1,NORTHPORT,AL,TUSCALOOSA,HYDROELECTRIC POWER GENERATION,53.9,53.0,53.0,0.0,0.0,1,0,0,WAT,NOT AVAILABLE,0,0,0,139170.00
2,BUCKS,AL,MOBILE,FOSSIL FUEL ELECTRIC POWER GENERATION,2569.5,2399.7,2430.7,774.0,272.0,10,2,1,NG,BIT,1292499,54793583,0,10499145.97
3,WETUMPKA,AL,ELMORE,HYDROELECTRIC POWER GENERATION,225.0,224.1,224.1,0.0,0.0,3,0,0,WAT,NOT AVAILABLE,0,0,0,554613.00
4,GADSDEN,AL,ETOWAH,FOSSIL FUEL ELECTRIC POWER GENERATION,138.0,130.0,130.0,0.0,0.0,2,0,0,NG,NOT AVAILABLE,0,697629,0,50435.00


In [96]:
# Init Respiratory Mortality Rate df (resp_df)
resp_df = pd.read_excel("data/RESP_DISEASE_MORTALITY_1980_2014.XLSX",header=1)
resp_df.dropna(inplace=True)
resp_df['Location'] = resp_df['Location'].str.upper()
resp_df = resp_df[['Location','Mortality Rate, 2000*','Mortality Rate, 2010*']]
resp_df.columns = ['Location','Deaths per 100,000 in 2000','Deaths per 100,000 in 2010']
resp_df["County"] = resp_df["Location"].str.split(", ",expand=True)[0]
resp_df["State"]= resp_df["Location"].str.split(", ",expand=True)[1]
resp_df = resp_df.drop('Location',axis = 1)
resp_df = resp_df[['County','State','Deaths per 100,000 in 2000','Deaths per 100,000 in 2010']].dropna().reset_index(drop=True)
resp_df['State'] = resp_df['State'].map(us_state_to_abbrev)
resp_df.head()

,County,State,"Deaths per 100,000 in 2000","Deaths per 100,000 in 2010"
0,AUTAUGA COUNTY,AL,"71.77 (68.27, 75.56)","77.75 (73.63, 81.85)"
1,BALDWIN COUNTY,AL,"52.07 (49.85, 54.48)","55.12 (52.87, 57.56)"
2,BARBOUR COUNTY,AL,"65.19 (61.87, 68.69)","67.52 (63.26, 71.55)"
3,BIBB COUNTY,AL,"74.13 (69.86, 78.72)","80.51 (75.82, 85.85)"
4,BLOUNT COUNTY,AL,"71.95 (68.40, 75.50)","80.48 (76.67, 84.46)"


In [116]:
loc_list = resp_df.head(0).copy()
# for county in power_df['COUNTY']:
#     loc_list.concat(resp_df.loc[resp_df['Location'].str.contains(county,case=False,regex=True),['Location']].reset_index(drop=True))



## todo: loop logic
# for each county in power_df -> locate in resp_df -> make sure the states match -> append to new dataframe
for idx,row in power_df.iterrows():
    #print(row['COUNTY'],row['STATE'])
    loc_list = pd.concat([loc_list,resp_df.loc[(resp_df['County'].str.contains(row['COUNTY'],case=False,regex=True)) & (resp_df['State'].str.contains(row['STATE'],case=False,regex=True))].reset_index(drop=True)],axis=1)


loc_list

In [115]:
loc_list = resp_df.head(0).copy()

x1 = resp_df.loc[(resp_df['County'].str.contains('TUSCALOOSA',case=False,regex=True)) & (resp_df['State'].str.contains('AL',case=False,regex=True))].reset_index(drop=True)
loc_list = pd.concat([loc_list,x1])
x2 = resp_df.loc[(resp_df['County'].str.contains('ALEUTIANS EAST',case=False,regex=True)) & (resp_df['State'].str.contains('AK',case=False,regex=True))].reset_index(drop=True)
loc_list = pd.concat([loc_list,x2])
loc_list

,County,State,"Deaths per 100,000 in 2000","Deaths per 100,000 in 2010"
0,TUSCALOOSA COUNTY,AL,"64.67 (62.02, 67.51)","67.32 (64.66, 70.10)"
0,ALEUTIANS EAST BOROUGH,AK,"48.21 (41.35, 56.36)","35.45 (29.74, 42.07)"


In [110]:
power_df.head(3)

,CITY,STATE,COUNTY,NAICS_DESC,OPER_CAP,SUMMER_CAP,WINTER_CAP,PLAN_CAP,RETIRE_CAP,GEN_UNITS,PLAN_UNIT,RETIR_UNIT,PRIM_FUEL,SEC_FUEL,COAL_USED,NGAS_USED,OIL_USED,NET_GEN
0,SAND POINT,AK,ALEUTIANS EAST,FOSSIL FUEL ELECTRIC POWER GENERATION,4.0,1.8,1.7,0.0,0.0,6,0,0,DFO,WND,0,0,0,347.00
1,NORTHPORT,AL,TUSCALOOSA,HYDROELECTRIC POWER GENERATION,53.9,53.0,53.0,0.0,0.0,1,0,0,WAT,NOT AVAILABLE,0,0,0,139170.00
2,BUCKS,AL,MOBILE,FOSSIL FUEL ELECTRIC POWER GENERATION,2569.5,2399.7,2430.7,774.0,272.0,10,2,1,NG,BIT,1292499,54793583,0,10499145.97


In [64]:
filtered_rsp = resp_df[resp_df['Location'].isin(loc_list)]

In [71]:
for idx,row in power_df.iterrows():
    print(row['COUNTY'],row['STATE'])

ALEUTIANS EAST AK
TUSCALOOSA AL
MOBILE AL
ELMORE AL
ETOWAH AL
WALKER AL
GREENE AL
CALHOUN AL
TUSCALOOSA AL
ELMORE AL
ST. CLAIR AL
CHILTON AL
ELMORE AL
CHILTON AL
WALKER AL
ELMORE AL
CHEROKEE AL
ELMORE AL
SHELBY AL
WATONWAN MN
WILCOX AL
LIMESTONE AL
COLBERT AL
MARSHALL AL
LAWRENCE AL
JACKSON AL
DE SOTO LA
COVINGTON AL
CLARK KY
COVINGTON AL
WASHINGTON AL
HALL NE
ADAMS NE
JUNEAU AK
JUNEAU AK
JUNEAU AK
JUNEAU AK
SKAGWAY AK
HAINES AK
KETCHIKAN GATEWAY AK
KODIAK ISLAND AK
ANCHORAGE AK
MATANUSKA-SUSITNA AK
JUNEAU AK
FAIRBANKS NORTH STAR AK
KETCHIKAN GATEWAY AK
KETCHIKAN GATEWAY AK
ULSTER NY
NOME AK
PETERSBURG AK
KENAI PENINSULA AK
SITKA AK
WRANGELL AK
KENAI PENINSULA AK
FINNEY KS
DILLINGHAM AK
STANLY NC
BERKELEY SC
PUTNAM FL
MONROE FL
MAYES OK
GARLAND AR
PULASKI AR
ST. FRANCIS AR
LAFAYETTE AR
HOT SPRING AR
PULASKI AR
MISSISSIPPI AR
PHILLIPS AR
HOT SPRING AR
CARROLL AR
CLARK AR
CLEBURNE AR
GREENE AR
CLAY AR
PIKE AR
BAXTER AR
GARLAND AR
BAXTER AR
YELL AR
FRANKLIN AR
WOODRUFF AR
OUACHITA AR
DE W